In [31]:
import subprocess
subprocess.run(['sudo','cpupower','frequency-set','-u','6.0Ghz'],stdout=subprocess.DEVNULL,stderr=subprocess.DEVNULL)

CompletedProcess(args=['sudo', 'cpupower', 'frequency-set', '-u', '6.0Ghz'], returncode=0)

In [32]:
import torch
from torch.utils.data import Dataset,DataLoader
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder

In [33]:
df=pd.read_csv("https://raw.githubusercontent.com/Arannamoy/datasets/refs/heads/main/Medical%20Cost%20Personal%20Datasets/insurance.csv")


In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [35]:
train_df,test_df=train_test_split(df,test_size=0.25,random_state=42)

In [36]:
label_encoder={}
categorical_cols=['sex','smoker','region']
for col in categorical_cols:
    le=LabelEncoder()
    train_df[col]=le.fit_transform(train_df[col])
    test_df[col]=le.transform(test_df[col])

In [37]:
train_df.head(5)

,age,sex,bmi,children,smoker,region,charges
693,24,1,23.655,0,0,1,2352.96845
1297,28,0,26.510,2,0,2,4340.44090
634,51,1,39.700,1,0,3,9391.34600
1022,47,1,36.080,1,1,2,42211.13820
178,46,0,28.900,2,0,3,8823.27900


In [38]:
X_train=train_df.drop(columns=['charges'])
y_train=train_df['charges']
X_test=test_df.drop(columns=['charges'])
y_test=test_df['charges']

In [39]:
X_train.head(1)

,age,sex,bmi,children,smoker,region
693,24,1,23.655,0,0,1


In [40]:
y_train.head(1)

693    2352.96845
Name: charges, dtype: float64

In [41]:
scaler=StandardScaler()
X_train_scaler=scaler.fit_transform(X_train)
X_test_scaler=scaler.transform(X_test)

In [42]:
class InsuranceDateset(Dataset):
    def __init__(self,X_train,y_train):
        super(InsuranceDateset,self).__init__()
        self.X_train=X_train
        self.y_train=y_train
    def __len__(self):
        return len(self.X_train)
    
    def __getitem__(self, index):
        features=torch.tensor(self.X_train[index],dtype=torch.float32)
        target=torch.tensor(self.y_train.values[index],dtype=torch.float32)
        return features,target

In [43]:
dataset=InsuranceDateset(X_train_scaler,y_train)

In [44]:
dataloader=DataLoader(dataset,batch_size=320,shuffle=True)

In [45]:
for batch_index,(features,targets) in enumerate(dataloader):
    print(f"Batch {batch_index+1}:")
    print(f"Features: {features.shape}")
    print(f"Tragets {targets.shape}")
    


Batch 1:
Features: torch.Size([320, 6])
Tragets torch.Size([320])
Batch 2:
Features: torch.Size([320, 6])
Tragets torch.Size([320])
Batch 3:
Features: torch.Size([320, 6])
Tragets torch.Size([320])
Batch 4:
Features: torch.Size([43, 6])
Tragets torch.Size([43])


In [46]:
import torch.nn as nn

In [47]:
class SimpleLinearRegression(nn.Module):
    def __init__(self,input_size,hidden_size,output_size):
        super(SimpleLinearRegression,self).__init__()
        self.network=nn.Sequential(
            nn.Linear(input_size,hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size,output_size),
            nn.ReLU(),
            nn.Linear(output_size,1)
        )
    
    def forward(self,x):
        return self.network(x)

In [48]:
model=SimpleLinearRegression(X_train.shape[1],64,128)

In [49]:
import torch.optim as optim

In [50]:
criterion=nn.MSELoss()
lr=0.001
optimizer=optim.Adam(model.parameters(),lr)

In [51]:
epochs=5000

X_train_tensor=torch.tensor(X_train_scaler,dtype=torch.float32)
y_train_tensor=torch.tensor(y_train.values,dtype=torch.float32).view(-1,1)
X_test_tensor=torch.tensor(X_test_scaler,dtype=torch.float32)
y_test_tensor=torch.tensor(y_test.values,dtype=torch.float32).view(-1,1)

for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    result=model(X_train_tensor)
    loss=criterion(result,y_train_tensor)
    loss.backward()
    optimizer.step()

    if (epoch+1)%1000==0:
        print(f"Epoch {epoch}/{epochs}, Loss: {loss.item()}")

Epoch 999/5000, Loss: 35228492.0
Epoch 1999/5000, Loss: 26304108.0
Epoch 2999/5000, Loss: 21265042.0
Epoch 3999/5000, Loss: 19991392.0
Epoch 4999/5000, Loss: 19176540.0


In [52]:
subprocess.run(['sudo','cpupower','frequency-set','-u','2.8Ghz'],stdout=subprocess.DEVNULL,stderr=subprocess.DEVNULL)

CompletedProcess(args=['sudo', 'cpupower', 'frequency-set', '-u', '2.8Ghz'], returncode=0)